In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=fbe93b5d9862be5467cae3632f932f7860f746b42b31fd92606ac36655834668
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, count


In [5]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Flights Analysis") \
    .getOrCreate()

# Load the flights dataset
flights = spark.read.format("csv") \
.option("header", "true") \
    .option("inferSchema", "true") \
    .load("/content/departuredelays.csv")

In [6]:
# Create a temporary view of the DataFrame
flights.createOrReplaceTempView("flights")

In [7]:
# Perform analysis using Spark SQL
# Example 1: Average delay by origin airport

avg_delay_by_origin = spark.sql("""
    SELECT origin, AVG(delay) as avg_delay
    FROM flights
    GROUP BY origin
    ORDER BY avg_delay DESC
    LIMIT 10
""")

In [9]:
avg_delay_by_origin.show()

+------+------------------+
|origin|         avg_delay|
+------+------------------+
|   GUM| 33.87777777777778|
|   LSE|26.532467532467532|
|   MQT| 23.87012987012987|
|   EGE| 20.57012542759407|
|   ROA|19.885106382978723|
|   MDW|19.657658556043078|
|   BTV|  18.7246192893401|
|   ORD|18.588917606028524|
|   IAD| 18.40343803056027|
|   SCE| 17.91616766467066|
+------+------------------+



In [10]:
# Example 2: Total flights and average delay by day of week

flights_by_day = spark.sql("""
    SELECT
        date,
        COUNT(*) as total_flights,
        AVG(delay) as avg_delay
    FROM flights
    GROUP BY date
    ORDER BY date
    LIMIT 7
""")

In [11]:
flights_by_day.collect()

[Row(date=1010005, total_flights=1, avg_delay=-8.0),
 Row(date=1010010, total_flights=1, avg_delay=-6.0),
 Row(date=1010020, total_flights=2, avg_delay=-1.0),
 Row(date=1010023, total_flights=1, avg_delay=14.0),
 Row(date=1010025, total_flights=2, avg_delay=15.0),
 Row(date=1010029, total_flights=1, avg_delay=49.0),
 Row(date=1010030, total_flights=3, avg_delay=-5.666666666666667)]

In [12]:
flights_by_day.show()

+-------+-------------+------------------+
|   date|total_flights|         avg_delay|
+-------+-------------+------------------+
|1010005|            1|              -8.0|
|1010010|            1|              -6.0|
|1010020|            2|              -1.0|
|1010023|            1|              14.0|
|1010025|            2|              15.0|
|1010029|            1|              49.0|
|1010030|            3|-5.666666666666667|
+-------+-------------+------------------+



In [13]:
# Example 3: Top 5 routes with the highest total delay

top_delayed_routes = spark.sql("""
    SELECT
        origin,
        destination,
        SUM(delay) as total_delay,
        COUNT(*) as flight_count
    FROM flights
    GROUP BY origin, destination
    ORDER BY total_delay DESC
    LIMIT 5
""")

In [14]:
top_delayed_routes.show()

+------+-----------+-----------+------------+
|origin|destination|total_delay|flight_count|
+------+-----------+-----------+------------+
|   LAX|        SFO|      51844|        3198|
|   ORD|        SFO|      41653|        1731|
|   SFO|        LAX|      40798|        3232|
|   LGA|        ATL|      35761|        2500|
|   JFK|        LAX|      35755|        2720|
+------+-----------+-----------+------------+



In [15]:
# Show results
print("Top 10 origins by average delay:")
avg_delay_by_origin.show()

print("\nFlights and average delay by day (first week):")
flights_by_day.show()

print("\nTop 5 routes with highest total delay:")
top_delayed_routes.show()



Top 10 origins by average delay:
+------+------------------+
|origin|         avg_delay|
+------+------------------+
|   GUM| 33.87777777777778|
|   LSE|26.532467532467532|
|   MQT| 23.87012987012987|
|   EGE| 20.57012542759407|
|   ROA|19.885106382978723|
|   MDW|19.657658556043078|
|   BTV|  18.7246192893401|
|   ORD|18.588917606028524|
|   IAD| 18.40343803056027|
|   SCE| 17.91616766467066|
+------+------------------+


Flights and average delay by day (first week):
+-------+-------------+------------------+
|   date|total_flights|         avg_delay|
+-------+-------------+------------------+
|1010005|            1|              -8.0|
|1010010|            1|              -6.0|
|1010020|            2|              -1.0|
|1010023|            1|              14.0|
|1010025|            2|              15.0|
|1010029|            1|              49.0|
|1010030|            3|-5.666666666666667|
+-------+-------------+------------------+


Top 5 routes with highest total delay:
+------+----

In [16]:
# Stop the SparkSession
spark.stop()

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, count, hour, month

In [35]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Extended Flights Analysis") \
    .getOrCreate()

# Load the flights dataset
flights = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/content/departuredelays.csv")



In [36]:
# Create a temporary view of the DataFrame
flights.createOrReplaceTempView("flights")



In [37]:
# Example 4: Average delay by origin airport (top 10)
avg_delay_by_origin = spark.sql("""
    SELECT origin, AVG(delay) as avg_delay
    FROM flights
    GROUP BY origin
    ORDER BY avg_delay DESC
    LIMIT 10
""")


In [38]:
# Example 5: Top 5 busiest routes
busiest_routes = spark.sql("""
    SELECT
        origin,
        destination,
        COUNT(*) as flight_count
    FROM flights
    GROUP BY origin, destination
    ORDER BY flight_count DESC
    LIMIT 5
""")


In [39]:
# Example 6: Monthly flight trends

monthly_trends = spark.sql("""
    SELECT
        SUBSTRING(CAST(date AS STRING), 5, 2) as month,
        COUNT(*) as total_flights,
        AVG(delay) as avg_delay
    FROM flights
    GROUP BY SUBSTRING(CAST(date AS STRING), 5, 2)
    ORDER BY month
""")


In [40]:
# Example 7: Percentage of delayed flights by origin
delayed_percentage = spark.sql("""
    SELECT
        origin,
        COUNT(*) as total_flights,
        SUM(CASE WHEN delay > 0 THEN 1 ELSE 0 END) as delayed_flights,
        (SUM(CASE WHEN delay > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) as delayed_percentage
    FROM flights
    GROUP BY origin
    ORDER BY delayed_percentage DESC
    LIMIT 10
""")



In [41]:
# Example 8: Average delay by hour of day
delay_by_hour = spark.sql("""
    SELECT
        CAST(SUBSTRING(CAST(date AS STRING), 10, 2) AS INT) as hour,
        AVG(delay) as avg_delay
    FROM flights
    GROUP BY SUBSTRING(CAST(date AS STRING), 10, 2)
    ORDER BY hour
""")


In [42]:
# Show results
print("Top 10 origins by average delay:")
avg_delay_by_origin.show()

print("\nTop 5 busiest routes:")
busiest_routes.show()

print("\nMonthly flight trends:")
monthly_trends.show()

print("\nTop 10 origins by percentage of delayed flights:")
delayed_percentage.show()

print("\nAverage delay by hour of day:")
delay_by_hour.show()




Top 10 origins by average delay:
+------+------------------+
|origin|         avg_delay|
+------+------------------+
|   GUM| 33.87777777777778|
|   LSE|26.532467532467532|
|   MQT| 23.87012987012987|
|   EGE| 20.57012542759407|
|   ROA|19.885106382978723|
|   MDW|19.657658556043078|
|   BTV|  18.7246192893401|
|   ORD|18.588917606028524|
|   IAD| 18.40343803056027|
|   SCE| 17.91616766467066|
+------+------------------+


Top 5 busiest routes:
+------+-----------+------------+
|origin|destination|flight_count|
+------+-----------+------------+
|   SFO|        LAX|        3232|
|   LAX|        SFO|        3198|
|   LAS|        LAX|        3016|
|   LAX|        LAS|        2964|
|   JFK|        LAX|        2720|
+------+-----------+------------+


Monthly flight trends:
+-----+-------------+------------------+
|month|total_flights|         avg_delay|
+-----+-------------+------------------+
|   00|        30649|11.788965382231067|
|   01|        25780|11.812063615205586|
|   02|        

In [43]:
# Stop the SparkSession
spark.stop()
